In [1]:
import sys
PROJ_ROOT = "."
sys.path.insert(0, PROJ_ROOT)
import os
os.chdir("../..")
os.environ['CUDA_VISIBLE_DEVICES'] = '4'

from core.gdrn_modeling.main_gdrn import setup

from core.utils.default_args_setup import my_default_argument_parser, my_default_setup

from tqdm import tqdm
from torch.cuda.amp import autocast
from torch.cuda.amp import autocast, GradScaler

from detectron2.data import MetadataCatalog
from core.gdrn_modeling.data_loader import build_gdrn_train_loader, build_gdrn_test_loader

import numpy as np
import matplotlib.pyplot as plt


/home/tianyou_zhang/.conda/envs/pytorch1.6/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


You requested to import horovod which is missing or not supported for your OS.
You requested to import horovod which is missing or not supported for your OS.


In [2]:
# args = my_default_argument_parser().parse_args(['--config-file', 'output/gdrn/lm/a6_cPnP_lm13_norm_14/a6_cPnP_lm13_norm.py', '--num-gpus', '1'])
# args = my_default_argument_parser().parse_args(['--config-file', 'configs/gdrn/ycbv/a6_cPnP_AugAAETrunc_BG0.5_Rsym_ycbv_real_pbr_visib20_10e_norm.py', '--num-gpus', '1'])
args = my_default_argument_parser().parse_args(['--config-file', '/home/tianyou_zhang/Project/NVR-Net/gdr-normal-NVR-Net/output/a6_cPnP_airplane_norm_23_debug/airplane.py', '--num-gpus', '1'])

cfg = setup(args)
# cfg.OUTPUT_DIR = 'debug/gdrn/ycbv/a6_cPnP_AugAAETrunc_BG0.5_Rsym_ycbv_real_pbr_visib20_10e_norm_test'
cfg.OUTPUT_DIR = 'debug/gdrn/airplane/a6_cPnP_AugAAETrunc_BG0.5_lmo_real_pbr0.1_40e_norm_efficientpose/'
cfg.TEST.USE_SVD = False

[1019_023254@core/gdrn_modeling/main_gdrn.py:72] optimizer_cfg: {'type': 'Ranger', 'lr': 0.0001, 'weight_decay': 0}
[1019_023254@core/gdrn_modeling/datasets/airplane.py:114] DBG register dataset: airplane
[10/19 02:32:54 detectron2]: Rank of current process: 0. World size: 1


[10/19 02:32:54 detectron2]: Environment info:
----------------------  -----------------------------------------------------------------------------------------
sys.platform            linux
Python                  3.6.15 | packaged by conda-forge | (default, Dec  3 2021, 18:49:41) [GCC 9.4.0]
numpy                   1.19.2
detectron2              0.3 @/home/tianyou_zhang/Project/detectron2/detectron2
Compiler                GCC 7.5
CUDA compiler           CUDA 11.7
detectron2 arch flags   7.5
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.6.0 @/home/tianyou_zhang/.conda/envs/pytorch1.6/lib/python3.6/site-packages/torch
PyTorch debug build     False
GPU available           True
GPU 0                   NVIDIA GeForce RTX 2080 Ti (arch=7.5)
CUDA_HOME               /usr/local/cuda
Pillow                  8.4.0
torchvision             0.7.0 @/home/tianyou_zhang/.conda/envs/pytorch1.6/lib/python3.6/site-packages/torchvision
torchvision arch flags  3.5, 5.0, 6.0, 7.0, 7.5
fvcore

In [3]:
dataset_meta = MetadataCatalog.get(cfg.DATASETS.TRAIN)
train_dset_names = cfg.DATASETS.TRAIN[0]
print(train_dset_names)
data_loader = build_gdrn_test_loader(cfg, train_dset_names)
data_loader_iter = iter(data_loader)
# data = next(data_loader_iter)

airplane


100%|██████████| 900/900 [00:00<00:00, 78031.95it/s]

[1019_023257 core.utils.dataset_utils@130]: Loading detections for airplane from: /home/tianyou_zhang/Project/NVR-Net/gdr-normal-NVR-Net/datasets/airplane/you3/bbox.json
WRN [1019_023257 core.utils.dataset_utils@140]: pop the original annotations, load detections
WRN [1019_023257 core.utils.dataset_utils@112]: Removed 0 images with empty detections. 900 images left.
[1019_023257 core.gdrn_modeling.data_loader@176]: Serializing 900 elements to byte tensors and concatenating them all ...
[1019_023257 core.gdrn_modeling.data_loader@181]: Serialized dataset takes 0.45 MiB


In [6]:
from core.gdrn_modeling.models import GDRN
model, optimizer = eval(cfg.MODEL.CDPN.NAME).build_model_optimizer(cfg)

from core.utils.my_checkpoint import MyCheckpointer
from torch.cuda.amp import GradScaler
import core.utils.my_comm as comm
cfg.MODEL.WEIGHTS = '/home/tianyou_zhang/Project/NVR-Net/gdr-normal-NVR-Net/output/a6_cPnP_airplane_norm_23_debug/model_final.pth'
# cfg.MODEL.WEIGHTS = '/home/fgkun/projects/GDR-Net/output/gdrn/lmo/a6_cPnP_AugAAETrunc_BG0.5_lmo_real_pbr0.1_40e_norm_1/model_final.pth'
# cfg.MODEL.WEIGHTS = '/home/fgkun/projects/GDR-Net/output/gdrn/lm/a6_cPnP_lm13_norm_4/model_0102399.pth'
from core.utils import solver_utils
scheduler = solver_utils.build_lr_scheduler(cfg, optimizer, total_iters=1)
AMP_ON = cfg.SOLVER.AMP.ENABLED
grad_scaler = GradScaler()
checkpointer = MyCheckpointer(
        model,
        cfg.OUTPUT_DIR,
        optimizer=optimizer,
        scheduler=scheduler,
        gradscaler=grad_scaler,
        save_to_disk=comm.is_main_process(),
    )
start_iter = checkpointer.resume_or_load(cfg.MODEL.WEIGHTS, resume=True).get("iteration", -1) + 1

model.eval()

from core.gdrn_modeling.engine_utils import batch_data, batch_data_test
from detectron2.utils.events import EventStorage

output = []

KeyError: 'type'

In [5]:
for img_index, data in tqdm(enumerate(data_loader_iter)):
    batch = batch_data_test(cfg, data)
    with autocast(enabled=cfg.TEST.AMP_TEST):
        out_dict = model(
            batch["roi_img"],
            roi_classes=batch["roi_cls"],
            roi_cams=batch["roi_cam"],
            roi_whs=batch["roi_wh"],
            roi_centers=batch["roi_center"],
            resize_ratios=batch["resize_ratio"],
            roi_coord_2d=batch.get("roi_coord_2d", None),
            roi_extents=batch.get("roi_extent", None),
        )
    output.append(out_dict["trans"].cpu().numpy())

100it [00:02, 44.88it/s]


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 10.75 GiB total capacity; 5.93 GiB already allocated; 10.50 MiB free; 5.93 GiB reserved in total by PyTorch)

In [6]:
cfg.TEST.AMP_TEST

False

In [11]:
img_index 

899

In [7]:
output[0].type()

'torch.cuda.FloatTensor'

In [9]:
next(data_loader_iter)

StopIteration: 

## Rewrite .txt to .json

In [ ]:
import json

txt_file = "/home/tianyou_zhang/Project/NVR-Net/gdr-normal-NVR-Net/datasets/airplane/zuo3/bbox.txt"
json_file = "/home/tianyou_zhang/Project/NVR-Net/gdr-normal-NVR-Net/datasets/airplane/zuo3/bbox.json"

out_line = {}
with open(txt_file, 'r') as in_file:
    with open(json_file, 'w') as out_file:
        for line in in_file:
            in_line = line.strip('\n').split(' ')            
            id, _, left_x, _, top_y, _, w, _, h = in_line
            out_line["{:d}/{:d}".format(int(0), int(id))]=[{"obj_id": int(0), "bbox_est":[float(left_x), float(top_y), float(w), float(h)], "score": int(1)}]
            # json.loads(out_line)
        json.dump(out_line, out_file)

## Original

In [ ]:
dataset_meta = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
train_dset_names = cfg.DATASETS.TRAIN
data_loader = build_gdrn_train_loader(cfg, train_dset_names)
data_loader_iter = iter(data_loader)
    
train_2_dset_names = cfg.DATASETS.get("TRAIN2", ())
train_2_ratio = cfg.DATASETS.get("TRAIN2_RATIO", 0.0)
if train_2_ratio > 0.0 and len(train_2_dset_names) > 0:
    data_loader_2 = build_gdrn_train_loader(cfg, train_2_dset_names)
    data_loader_2_iter = iter(data_loader_2)
else:
    data_loader_2 = None
    data_loader_2_iter = None

100%|██████████| 900/900 [00:00<00:00, 74714.46it/s]


AttributeError: 'str' object has no attribute 'get'

In [ ]:
if np.random.rand() < train_2_ratio:
    data = next(data_loader_2_iter)
else:
    data = next(data_loader_iter)

In [ ]:
# for i in range(10000):
#     data = next(data_loader_iter)

In [ ]:
# for i in range(10000):
#     data = next(data_loader_2_iter)

In [ ]:
roi_img = data[0]['roi_img'].cpu().numpy().transpose([1,2,0])
roi_mask_visib = data[0]['roi_mask_visib'].cpu().numpy()
roi_norm = data[0]['roi_norm'].cpu().numpy()
roi_stc_norm = roi_norm[:3].transpose([1,2,0])
roi_dyn_norm = roi_norm[3:].transpose([1,2,0])

In [ ]:
fig, axs = plt.subplots(24,4, figsize=(16,96))
for i,d in enumerate(data):
    roi_img = d['roi_img'].cpu().numpy().transpose([1,2,0])
    roi_mask_visib = d['roi_mask_visib'].cpu().numpy()
    roi_norm = d['roi_norm'].cpu().numpy()
    roi_stc_norm = roi_norm[:3].transpose([1,2,0])
    roi_dyn_norm = roi_norm[3:].transpose([1,2,0])
    axs[i,0].imshow(roi_img)
    axs[i,0].set_title('{}'.format(i))
    axs[i,1].imshow(roi_mask_visib)
    axs[i,2].imshow(roi_stc_norm)
    axs[i,3].imshow(roi_dyn_norm)

In [ ]:
[(i,d['file_name']) for i,d in enumerate(data)]

In [ ]:
i = 11
data[i].keys()

In [ ]:
def cal_R(stc, dyn, weight=None):
    if isinstance(weight, np.ndarray):
        weight = weight.reshape([-1])
        assert weight.shape[0] == stc.shape[0], 'weight size not match!'
        W = np.diag(weight)
    else:
        W = np.eye(stc.shape[0])
    AA = stc.T.dot(W).dot(dyn)
    U, S, VH = np.linalg.svd(AA)
    R_matrix = VH.T.dot(U.T)
    return R_matrix

def re(R_est, R_gt):
    from scipy.linalg import logm
    import numpy.linalg as LA
    assert (R_est.shape == R_gt.shape == (3, 3))
    temp = logm(np.dot(np.transpose(R_est), R_gt))
    rd_rad = LA.norm(temp, 'fro') / np.sqrt(2)
    rd_deg = rd_rad / np.pi * 180
    return rd_deg

roi_mask_visib = data[i]['roi_mask_visib'].cpu().numpy()
roi_norm = data[i]['roi_norm'].cpu().numpy()
idx = np.where(roi_mask_visib>0)
# stc_norm = roi_norm[:3].transpose([1,2,0])[idx]
# dyn_norm = roi_norm[3:].transpose([1,2,0])[idx]
stc_norm = (roi_norm[:3].transpose([1,2,0])[idx]-0.5)*2
dyn_norm = (roi_norm[3:].transpose([1,2,0])[idx]-0.5)*2
norm_rot = cal_R(stc_norm, dyn_norm)
print(norm_rot)

part_allo_rot = data[i]['allo_rot6d'].cpu().numpy()
gt_allo_rot = np.zeros([3,3])
gt_allo_rot[:3,0] = part_allo_rot[:3]
gt_allo_rot[:3,1] = part_allo_rot[3:]
gt_allo_rot[:3,2] = np.cross(part_allo_rot[:3], part_allo_rot[3:])
print(re(gt_allo_rot, norm_rot))

In [ ]:
from core.utils.utils import egocentric_to_allocentric_gl, CV2GL
def SkewSymRotm(a,b,c):
    return np.array([
        [0, -c, b],
        [c, 0, -a],
        [-b, a, 0]
    ])

def SkewSymRotmv(u):
    a,b,c = u.reshape(3).tolist()
    return SkewSymRotm(a,b,c)

def Axis2Rotm(ang, axis):
    axis = -axis if ang < 0 else axis
    ang = -ang if ang < 0 else ang
    axis = axis/np.linalg.norm(axis)
    sw = SkewSymRotmv(axis)
    return np.eye(3)-sw*np.sin(ang)+sw.dot(sw)*(1-np.cos(ang))
def ang_rectify(trans, reverse=False):
    if np.linalg.norm(trans[:2]) < np.abs(trans[-1])/1e8:
        return np.eye(3)
    ang = np.arctan(np.linalg.norm(trans[:2]/trans[-1]))
    axis = np.array([-trans[1], trans[0], 0]) if trans[0]>=0 else np.array([trans[1], -trans[0], 0])
    theta = ang if trans[0]>=0 else -ang
    if reverse:
        return Axis2Rotm(theta, axis).T
    else:
        return Axis2Rotm(theta, axis)
CV2GL.dot(ang_rectify(data[i]['trans'])).dot(CV2GL).dot(data[i]['ego_rot'])

In [ ]:
ego_pose = np.zeros([3,4])
ego_pose[:3,:3] = data[i]['ego_rot']
ego_pose[:3,3] = data[i]['trans']
egocentric_to_allocentric_gl(ego_pose)

In [ ]:
data[i]['ego_rot']

In [ ]:
data[i]['allo_rot6d']

In [ ]:
from core.gdrn_modeling.models import GDRN
model, optimizer = eval(cfg.MODEL.CDPN.NAME).build_model_optimizer(cfg)

Ranger optimizer loaded. 
Gradient Centralization usage = True
GC applied to both conv and fc layers
[1019_005541 core.gdrn_modeling.models.GDRN@961]: load backbone weights from: torchvision://resnet18
[1019_005541 core.gdrn_modeling.models.GDRN@101]: Use load_from_torchvision loader
WRN [1019_005541 core.gdrn_modeling.models.GDRN@104]: The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



In [ ]:
from core.utils.my_checkpoint import MyCheckpointer
from torch.cuda.amp import GradScaler
import core.utils.my_comm as comm
cfg.MODEL.WEIGHTS = '/home/tianyou_zhang/Project/NVR-Net/gdr-normal-NVR-Net/output/a6_cPnP_airplane_norm_23_debug/model_final.pth'
# cfg.MODEL.WEIGHTS = '/home/fgkun/projects/GDR-Net/output/gdrn/lmo/a6_cPnP_AugAAETrunc_BG0.5_lmo_real_pbr0.1_40e_norm_1/model_final.pth'
# cfg.MODEL.WEIGHTS = '/home/fgkun/projects/GDR-Net/output/gdrn/lm/a6_cPnP_lm13_norm_4/model_0102399.pth'
from core.utils import solver_utils
scheduler = solver_utils.build_lr_scheduler(cfg, optimizer, total_iters=1)
grad_scaler = GradScaler()
checkpointer = MyCheckpointer(
        model,
        cfg.OUTPUT_DIR,
        optimizer=optimizer,
        scheduler=scheduler,
        gradscaler=grad_scaler,
        save_to_disk=comm.is_main_process(),
    )
start_iter = checkpointer.resume_or_load(cfg.MODEL.WEIGHTS, resume=True).get("iteration", -1) + 1

[10/19 00:55:45 fvcore.common.checkpoint]: [Checkpointer] Loading from /home/tianyou_zhang/Project/NVR-Net/gdr-normal-NVR-Net/output/a6_cPnP_airplane_norm_23_debug/model_final.pth ...


In [ ]:
from core.gdrn_modeling.engine_utils import batch_data, batch_data_test
from detectron2.utils.events import EventStorage
data = next(data_loader_iter)
with EventStorage(start_iter) as storage:
    batch = batch_data_test(cfg, data)

In [ ]:
print(batch)

{'im_H': tensor([1080.], device='cuda:0'), 'im_W': tensor([1418.], device='cuda:0'), 'roi_img': tensor([[[[0.7451, 0.7451, 0.7451,  ..., 0.7490, 0.7451, 0.7451],
          [0.7451, 0.7451, 0.7451,  ..., 0.7451, 0.7451, 0.7451],
          [0.7451, 0.7451, 0.7451,  ..., 0.7490, 0.7451, 0.7451],
          ...,
          [0.7412, 0.7373, 0.7373,  ..., 0.7373, 0.7412, 0.7412],
          [0.7373, 0.7373, 0.7373,  ..., 0.7412, 0.7412, 0.7412],
          [0.7373, 0.7373, 0.7373,  ..., 0.7412, 0.7412, 0.7412]],

         [[0.8118, 0.8157, 0.8118,  ..., 0.8157, 0.8157, 0.8118],
          [0.8118, 0.8118, 0.8118,  ..., 0.8118, 0.8118, 0.8118],
          [0.8118, 0.8118, 0.8118,  ..., 0.8157, 0.8157, 0.8118],
          ...,
          [0.8157, 0.8157, 0.8157,  ..., 0.8157, 0.8157, 0.8157],
          [0.8157, 0.8118, 0.8157,  ..., 0.8157, 0.8157, 0.8157],
          [0.8157, 0.8118, 0.8157,  ..., 0.8157, 0.8157, 0.8157]],

         [[0.8745, 0.8784, 0.8745,  ..., 0.8784, 0.8745, 0.8745],
          [0

In [ ]:

out_dict = model(
    batch["roi_img"],
    roi_classes=batch["roi_cls"],
    roi_cams=batch["roi_cam"],
    roi_whs=batch["roi_wh"],
    roi_centers=batch["roi_center"],
    resize_ratios=batch["resize_ratio"],
    # roi_coord_2d=None,
    # roi_extents=None,
    roi_coord_2d=batch.get("roi_coord_2d", None),
    roi_extents=batch.get("roi_extent", None),
)

In [ ]:
out_dict

{'rot': tensor([[[ 0.9240, -0.3541, -0.1442],
          [ 0.1335, -0.0545,  0.9895],
          [-0.3582, -0.9336, -0.0031]]]),
 'trans': tensor([[-1.4722e-07, -2.5213e-06,  3.8854e+00]], device='cuda:0',
        grad_fn=<CatBackward>)}

In [ ]:
from core.gdrn_modeling.engine_utils import batch_data
from detectron2.utils.events import EventStorage
with EventStorage(start_iter) as storage:
#     if np.random.rand() < train_2_ratio:
#         data = next(data_loader_2_iter)
#     else:
#         data = next(data_loader_iter)
    batch = batch_data(cfg, data)
    out_dict = model(
        batch["roi_img"],
        gt_norm=batch.get("roi_norm", None),
        gt_norm_bin=batch.get("roi_norm_bin", None),
        gt_mask_trunc=batch["roi_mask_trunc"],
        gt_mask_visib=batch["roi_mask_visib"],
        gt_mask_obj=batch["roi_mask_obj"],
        gt_region=batch.get("roi_region", None),
        gt_allo_quat=batch.get("allo_quat", None),
        gt_ego_quat=batch.get("ego_quat", None),
        gt_allo_rot6d=batch.get("allo_rot6d", None),
        gt_ego_rot6d=batch.get("ego_rot6d", None),
        gt_allo_rot=batch.get("allo_rot", None),
        gt_ego_rot=batch.get("ego_rot", None),
        gt_trans=batch.get("trans", None),
        gt_trans_ratio=batch.get(["roi_trans_ratio"],None),
        gt_points=batch.get("roi_points", None),
        sym_infos=batch.get("sym_info", None),
        roi_classes=batch["roi_cls"],
        roi_cams=batch["roi_cam"],
        roi_whs=batch["roi_wh"],
        roi_centers=batch["roi_center"],
        resize_ratios=batch["resize_ratio"],
        roi_coord_2d=batch.get("roi_coord_2d", None),
        roi_extents=batch.get("roi_extent", None),
        do_loss=False,
    )

KeyError: 'trans_ratio'

In [ ]:
out_dict.keys()

dict_keys(['rot', 'trans'])

In [ ]:
# fig, axs = plt.subplots(1,3,figsize=(30,10))
# axs[0].imshow(batch["roi_mask_trunc"][0].cpu().numpy())
# axs[1].imshow(batch["roi_mask_visib"][0].cpu().numpy())
# axs[2].imshow(batch["roi_mask_obj"][0].cpu().numpy())
ii = 6
plt.imshow(np.stack([batch["roi_mask_trunc"][ii].cpu().numpy(), batch["roi_mask_visib"][ii].cpu().numpy(), batch["roi_mask_obj"][ii].cpu().numpy()], axis=0).transpose([1,2,0]))

In [ ]:
from matplotlib.colors import Normalize
norm = Normalize(vmin = -1, vmax = 1)
gt_mask_numpy = batch["roi_mask_visib"][i].cpu().numpy()
pred_norm_stc_x = gt_mask_numpy*out_dict['norm_stc_x'][i].detach().cpu().numpy()
pred_norm_stc_y = gt_mask_numpy*out_dict['norm_stc_y'][i].detach().cpu().numpy()
pred_norm_stc_z = gt_mask_numpy*out_dict['norm_stc_z'][i].detach().cpu().numpy()
pred_norm_stc = np.concatenate([pred_norm_stc_x, pred_norm_stc_y, pred_norm_stc_z], axis=0)
pred_norm_stc = pred_norm_stc.transpose([1,2,0])
pred_norm_dyn_x = gt_mask_numpy*out_dict['norm_dyn_x'][i].detach().cpu().numpy()
pred_norm_dyn_y = gt_mask_numpy*out_dict['norm_dyn_y'][i].detach().cpu().numpy()
pred_norm_dyn_z = gt_mask_numpy*out_dict['norm_dyn_z'][i].detach().cpu().numpy()
pred_norm_dyn = np.concatenate([pred_norm_dyn_x, pred_norm_dyn_y, pred_norm_dyn_z], axis=0)
pred_norm_dyn = pred_norm_dyn.transpose([1,2,0])
fig, axs = plt.subplots(4,4, figsize=(20,20))


gt_stc_norm = roi_norm[:3].transpose([1,2,0])
roi_norm_np = batch['roi_norm'][i].cpu().numpy()
gt_stc_norm = roi_norm_np[:3].transpose([1,2,0])
gt_stc_norm_x = gt_stc_norm[:,:,0]
gt_stc_norm_y = gt_stc_norm[:,:,1]
gt_stc_norm_z = gt_stc_norm[:,:,2]
gt_dyn_norm = roi_norm_np[3:].transpose([1,2,0])
gt_dyn_norm_x = gt_dyn_norm[:,:,0]
gt_dyn_norm_y = gt_dyn_norm[:,:,1]
gt_dyn_norm_z = gt_dyn_norm[:,:,2]

axs[0,0].imshow(pred_norm_stc, norm=norm)
axs[0,1].imshow(gt_stc_norm, norm=norm)
axs[0,2].imshow(pred_norm_dyn, norm=norm)
axs[0,3].imshow(gt_dyn_norm, norm=norm)

axs[1,0].imshow(pred_norm_stc_x[0], norm=norm)
axs[1,1].imshow(gt_stc_norm_x, norm=norm)
axs[1,2].imshow(pred_norm_dyn_x[0], norm=norm)
axs[1,3].imshow(gt_dyn_norm_x, norm=norm)

axs[2,0].imshow(pred_norm_stc_y[0], norm=norm)
axs[2,1].imshow(gt_stc_norm_y, norm=norm)
axs[2,2].imshow(pred_norm_dyn_y[0], norm=norm)
axs[2,3].imshow(gt_dyn_norm_y, norm=norm)

axs[3,0].imshow(pred_norm_stc_z[0], norm=norm)
axs[3,1].imshow(gt_stc_norm_z, norm=norm)
axs[3,2].imshow(pred_norm_dyn_z[0], norm=norm)
axs[3,3].imshow(gt_dyn_norm_z, norm=norm)

In [ ]:
idx = np.where(gt_mask_numpy > 0)
# pred_stc_norm = pred_norm_stc[idx]
# pred_dyn_norm = pred_norm_dyn[idx]
pred_stc_norm = (pred_norm_stc[idx]-0.5)*2
pred_dyn_norm = (pred_norm_dyn[idx]-0.5)*2
cal_R(pred_stc_norm, pred_dyn_norm)

In [ ]:
out_dict['rot'][i]

In [ ]:
batch['allo_rot6d'][i]

In [ ]:
allo_pose_numpy = batch['allo_rot'][i].cpu().numpy()
gt_stc_norm_i = (gt_stc_norm-0.5)*2
gt_dyn_norm_i = (gt_dyn_norm-0.5)*2
tmp_idx = np.where(np.linalg.norm(gt_stc_norm_i, axis=-1)>0.5)
gt_stc_norm_vec = gt_stc_norm_i[tmp_idx]
gt_dyn_norm_vec = gt_dyn_norm_i[tmp_idx]

In [ ]:
np.sum(np.abs((allo_pose_numpy.dot(gt_stc_norm_vec.T)).T-gt_dyn_norm_vec))/np.shape(gt_stc_norm_vec)[0]

In [ ]:
import torch.nn as nn
import detectron2.utils.comm as comm
gt_mask_norm = batch["roi_mask_visib"]
print(nn.L1Loss(reduction="sum")(out_dict['norm_stc_x']*gt_mask_norm[:, None], batch['roi_norm'][:,:1]*gt_mask_norm[:, None]) / gt_mask_norm.sum().float().clamp(min=1.0))
print(nn.L1Loss(reduction="sum")(out_dict['norm_stc_y']*gt_mask_norm[:, None], batch['roi_norm'][:,1:2]*gt_mask_norm[:, None]) / gt_mask_norm.sum().float().clamp(min=1.0))
print(nn.L1Loss(reduction="sum")(out_dict['norm_stc_z']*gt_mask_norm[:, None], batch['roi_norm'][:,2:3]*gt_mask_norm[:, None]) / gt_mask_norm.sum().float().clamp(min=1.0))

In [ ]:
import torch
def norm_pose_loss_dyn(base_loss, normal_stc, normal_dyn, allo_pose):
    shape = normal_stc.size()
    normal_dyn_pose = torch.bmm(allo_pose, (normal_stc.reshape([shape[0], 3,-1]))).reshape(shape)
    loss = base_loss(normal_dyn, normal_dyn_pose.detach())
    return loss

def norm_pose_loss_stc(base_loss, normal_stc, normal_dyn, allo_pose):
    shape = normal_stc.size()
    normal_dyn_pose = torch.bmm(allo_pose, (normal_stc.reshape([shape[0], 3,-1]))).reshape(shape)
    loss = base_loss(normal_dyn_pose, normal_dyn.detach())
    return loss
gt_mask_norm = batch["roi_mask_visib"]
out_stc_norm = torch.cat([(out_dict['norm_stc_x']-0.5)*2 * gt_mask_norm[:, None], (out_dict['norm_stc_y']-0.5)*2 * gt_mask_norm[:, None], (out_dict['norm_stc_z']-0.5)*2 * gt_mask_norm[:, None]], dim=1)
out_dyn_norm = torch.cat([(out_dict['norm_dyn_x']-0.5)*2 * gt_mask_norm[:, None], (out_dict['norm_dyn_y']-0.5)*2 * gt_mask_norm[:, None], (out_dict['norm_dyn_z']-0.5)*2 * gt_mask_norm[:, None]], dim=1)
gt_stc_norm = (batch['roi_norm'][:, :3]-0.5)*2
gt_dyn_norm = (batch['roi_norm'][:, 3:]-0.5)*2
print(norm_pose_loss_stc(nn.L1Loss(reduction="sum"), out_stc_norm, out_dyn_norm, batch['allo_rot']) / gt_mask_norm.sum().float().clamp(min=1.0))
print(norm_pose_loss_dyn(nn.L1Loss(reduction="sum"), out_stc_norm, out_dyn_norm, batch['allo_rot']) / gt_mask_norm.sum().float().clamp(min=1.0))
print(norm_pose_loss_stc(nn.L1Loss(reduction="sum"), gt_stc_norm, gt_dyn_norm, batch['allo_rot']) / gt_mask_norm.sum().float().clamp(min=1.0))
print(norm_pose_loss_dyn(nn.L1Loss(reduction="sum"), gt_stc_norm, gt_dyn_norm, batch['allo_rot']) / gt_mask_norm.sum().float().clamp(min=1.0))

In [ ]:
gt_mask_norm = batch["roi_mask_visib"]
out_stc_norm = torch.cat([(out_dict['norm_stc_x']-0.5)*2 * gt_mask_norm[:, None], (out_dict['norm_stc_y']-0.5)*2 * gt_mask_norm[:, None], (out_dict['norm_stc_z']-0.5)*2 * gt_mask_norm[:, None]], dim=1)
out_dyn_norm = torch.cat([(out_dict['norm_dyn_x']-0.5)*2 * gt_mask_norm[:, None], (out_dict['norm_dyn_y']-0.5)*2 * gt_mask_norm[:, None], (out_dict['norm_dyn_z']-0.5)*2 * gt_mask_norm[:, None]], dim=1)
gt_stc_norm = (batch['roi_norm'][:, :3]-0.5)*2
gt_dyn_norm = (batch['roi_norm'][:, 3:]-0.5)*2
shape = out_stc_norm.size()
print(nn.L1Loss(reduction="sum")(torch.bmm(out_dict['rot_allo'], out_stc_norm.reshape([shape[0], 3,-1])), out_dyn_norm.reshape([shape[0], 3,-1])) / gt_mask_norm.sum().float().clamp(min=1.0))
print(nn.L1Loss(reduction="sum")(torch.bmm(out_dict['rot_allo'], gt_stc_norm.reshape([shape[0], 3,-1])), gt_dyn_norm.reshape([shape[0], 3,-1])) / gt_mask_norm.sum().float().clamp(min=1.0))
print(nn.L1Loss(reduction="sum")(torch.bmm(batch['allo_rot'], gt_stc_norm.reshape([shape[0], 3,-1])), gt_dyn_norm.reshape([shape[0], 3,-1])) / gt_mask_norm.sum().float().clamp(min=1.0))

In [ ]:
iii = 13
gt_mask_norm_np = gt_mask_norm[iii].cpu().numpy()
pred_norm_stc_x_np = out_dict['norm_stc_x'][iii].detach().cpu().numpy()
gt_norm_stc_x_np = batch['roi_norm'][iii,:1].cpu().numpy()
np.mean(np.abs(pred_norm_stc_x_np-gt_norm_stc_x_np)[0][np.where(gt_mask_norm_np>0)])

In [ ]:
gt_norm_stc_x_np.shape

In [ ]:
pred_norm_stc_x_np.shape

In [ ]:
plt.imshow(out_dict['mask'].detach().cpu().numpy()[i,0])

In [ ]:
_, axs = plt.subplots(1 ,2, figsize=(10,5))
axs[0].imshow(np.argmax(out_dict['region'].detach().cpu().numpy()[i], axis=0)*gt_mask_norm.cpu().numpy()[i])
axs[1].imshow(batch['roi_region'].cpu().numpy()[i])

In [ ]:
with EventStorage(start_iter) as storage:
    _, loss_dict = model(
        batch["roi_img"],
        gt_norm=batch.get("roi_norm", None),
        gt_norm_bin=batch.get("roi_norm_bin", None),
        gt_mask_trunc=batch["roi_mask_trunc"],
        gt_mask_visib=batch["roi_mask_visib"],
        gt_mask_obj=batch["roi_mask_obj"],
        gt_region=batch.get("roi_region", None),
        gt_allo_quat=batch.get("allo_quat", None),
        gt_ego_quat=batch.get("ego_quat", None),
        gt_allo_rot6d=batch.get("allo_rot6d", None),
        gt_ego_rot6d=batch.get("ego_rot6d", None),
        gt_allo_rot=batch.get("allo_rot", None),
        gt_ego_rot=batch.get("ego_rot", None),
        gt_trans=batch.get("trans", None),
        gt_trans_ratio=batch["roi_trans_ratio"],
        gt_points=batch.get("roi_points", None),
        sym_infos=batch.get("sym_info", None),
        roi_classes=batch["roi_cls"],
        roi_cams=batch["roi_cam"],
        roi_whs=batch["roi_wh"],
        roi_centers=batch["roi_center"],
        resize_ratios=batch["resize_ratio"],
        roi_coord_2d=batch.get("roi_coord_2d", None),
        roi_extents=batch.get("roi_extent", None),
        num_epoch=160,
        do_loss=True,
    )

In [ ]:
loss_dict

In [ ]:
cfg.MODEL.CDPN.ROT_HEAD

In [ ]:
model